# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [8]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [9]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [10]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    X_tr = X_train.copy()
    X_va = X_val.copy()
    if hyperparams['use_pca']:
        if hyperparams['pca_standardize']:
            scaler_pca = StandardScaler()
            X_tr = scaler_pca.fit_transform(X_tr)
            X_va = scaler_pca.transform(X_va)
        pca = PCA(n_components=hyperparams['pca_components'])
        X_tr = pca.fit_transform(X_tr)
        X_va = pca.transform(X_va)

    if hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_tr = scaler.fit_transform(X_tr)
        X_va = scaler.transform(X_va)

    # Aggiunge il termine costante ai dati
    X_tr_bias = np.hstack([np.ones((X_tr.shape[0], 1)), X_tr])
    X_va_bias = np.hstack([np.ones((X_va.shape[0], 1)), X_va])


    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        lam = hyperparams['reg_lambda']
        I = np.eye(X_tr_bias.shape[1])
        I[0, 0] = 0  
        A = X_tr_bias.T @ X_tr_bias + lam * I
        b = X_tr_bias.T @ y_train
        w = np.linalg.solve(A, b)
    else:
        w = np.linalg.solve(X_tr_bias.T @ X_tr_bias, X_tr_bias.T @ y_train)

    # Calcolo predizioni
    y_tr_pred = X_tr_bias @ w
    y_va_pred = X_va_bias @ w

    # Calcola il MAE
    mae_tr = np.mean(np.abs(y_train - y_tr_pred))
    mae_va = np.mean(np.abs(y_val - y_va_pred))

    return w, mae_tr, mae_va

# Dividi il dataset in training e test set
X_sh, y_sh = shuffle(X.values, y.values, random_state=42)
n = X_sh.shape[0]
n_train_val = int(0.8 * n)
X_train_val, y_train_val = X_sh[:n_train_val], y_sh[:n_train_val]
X_test, y_test = X_sh[n_train_val:], y_sh[n_train_val:]

# Dividi il training set in training set effettivo e validation set
n_train = int(0.75 * n_train_val)
X_train, y_train = X_train_val[:n_train], y_train_val[:n_train]
X_val, y_val = X_train_val[n_train:], y_train_val[n_train:]

# Trova la configurazione di iperparametri migliore
best_cfg = None
best_val_mae = np.inf
best_train_mae = None
for cfg in configs:
    _, mae_tr, mae_va = pipeline(X_train, y_train, X_val, y_val, cfg)
    if mae_va < best_val_mae:
        best_val_mae = mae_va
        best_train_mae = mae_tr
        best_cfg = cfg

# Riallena il modello sul training set completo
X_full = np.vstack([X_train, X_val])
y_full = np.hstack([y_train, y_val])


# Calcola il MAE sul test set
_, mae_full_tr, mae_test = pipeline(X_full, y_full, X_test, y_test, best_cfg)

# Stampa  risultati
print("La migliore configurazione di iperparametri:\n", best_cfg)
print(f"Miglior MAE sul validation set: {best_val_mae:.4f}")
print(f"MAE sul training set corrispondente: {best_train_mae:.4f}")
print(f"MAE sul test set: {mae_test:.4f}")


/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


La migliore configurazione di iperparametri:
 {'use_pca': True, 'pca_standardize': True, 'pca_components': 10, 'data_standardize': True, 'use_regularization': False, 'reg_lambda': None}
Miglior MAE sul validation set: 0.5822
MAE sul training set corrispondente: 0.5854
MAE sul test set: 0.5958
